# Museum Analysis

This notebook connects to the museum database and conducts simple analysis of the data.  
- _Any additional information relating to the results of the analyses will be included with each individual query_


## Imports


In [1]:
import extract
from pipeline import get_cursor, get_connection
import os
import psycopg2


## Setup
- _Connecting to the database, and establishing a connection to it._

In [2]:

database_name = os.getenv("DATABASE_NAME")
database_username = os.getenv("DATABASE_USERNAME")
database_ip = os.getenv("DATABASE_IP")
database_password = os.getenv("DATABASE_PASSWORD")
database_port = os.getenv("DATABASE_PORT")

conn = psycopg2.connect(
    database=database_name,
    user=database_username,
    host=database_ip,
    password=database_password,
    port=database_port
)


## Data Queries

### What exhibition is most frequently visited?

In [3]:
with get_cursor(conn) as cursor:
    
    cursor.execute(
    f"""
    SELECT e.exhibition_name, COUNT(DISTINCT r.review_id) + COUNT(DISTINCT ar.assistance_request_id) as total
    FROM exhibition as e
    JOIN review AS r
    ON e.exhibition_id = r.exhibition_id
    INNER JOIN assistance_request AS ar
    ON e.exhibition_id = ar.exhibition_id
    GROUP BY e.exhibition_id
    ORDER BY total DESC
    LIMIT 1;
    """)

    most_review_and_ar = cursor.fetchone()

    cursor.execute(
    f"""
    SELECT e.exhibition_name, COUNT(DISTINCT r.review_id)
    FROM exhibition as e
    JOIN review AS r
    ON e.exhibition_id = r.exhibition_id
    INNER JOIN assistance_request AS ar
    ON e.exhibition_id = ar.exhibition_id
    GROUP BY e.exhibition_id
    ORDER BY e.exhibition_id DESC
    LIMIT 1;
    """)

    most_review = cursor.fetchone()

print(f"{most_review[0]} is the most frequently visited exhibition, with {most_review[1]} unique reviews")
print("However, factoring in assistance requests:")
print(f"{most_review_and_ar[0]} is the most frequently visited exhibition, with {most_review_and_ar[1]} unique reviews AND assistance requests.")

Thunder Lizards is the most frequently visited exhibition, with 2586 unique reviews
However, factoring in assistance requests:
Our Polluted World is the most frequently visited exhibition, with 2656 unique reviews AND assistance requests.


### What hour of the day has the most ratings?
- _Note: The highest rating is a 4_

In [4]:
with get_cursor(conn) as cursor:

    cursor.execute(
    f"""
    SELECT EXTRACT(HOUR FROM creation_date) as hour, COUNT(*)
    FROM review
    GROUP BY hour
    ORDER BY COUNT(*) DESC
    LIMIT 1;
    """)

    hour = cursor.fetchone()

answer = hour[0]
num_ratings = hour[1]
print(f"The hour of the day with the most ratings is {answer}:00-{answer}:59, with {num_ratings} ratings")

The hour of the day with the most ratings is 13:00-13:59, with 1679 ratings


### What is the average rating for each exhibition?

In [5]:
with get_cursor(conn) as cursor:

    cursor.execute(
    f"""
    SELECT e.exhibition_name, AVG(r.numeric_value)
    FROM exhibition AS e
    JOIN review AS rev
    ON e.exhibition_id = rev.exhibition_id
    JOIN rating AS r
    ON rev.rating_id = r.rating_id
    GROUP BY e.exhibition_name
    ORDER BY AVG(r.numeric_value) DESC;
    """)

    result = cursor.fetchall()

for row in result:
    exhibition_name = row[0]
    average_rating = round(row[1], 1)
    print(f"{exhibition_name} has an average rating of {average_rating}/{float(4)}")

Thunder Lizards has an average rating of 2.9/4.0
Cetacean Sensations has an average rating of 2.1/4.0
Adaptation has an average rating of 2.0/4.0
Measureless to Man has an average rating of 2.0/4.0
The Crenshaw Collection has an average rating of 1.6/4.0
Our Polluted World has an average rating of 1.2/4.0


### What proportion of all 3+ ratings are given to exhibition 4 (_Our Polluted World_)?


In [6]:
with get_cursor(conn) as cursor:

    cursor.execute(
    f"""
    SELECT e.exhibition_name, COUNT(r.numeric_value) as rate
    FROM exhibition AS e
    JOIN review AS rev
    ON e.exhibition_id = rev.exhibition_id
    JOIN rating AS r
    ON rev.rating_id = r.rating_id
    WHERE r.numeric_value in (3,4)
    GROUP BY e.exhibition_name;
    """)

    result = cursor.fetchall()

total_ratings = 0
for row in result:
    total_ratings += row[1]
exhibition_four_proportion = (result[3][1] / total_ratings) * 100

print(f"{round(exhibition_four_proportion, 1)}% of all 3+ ratings were given to '{result[3][0]}'")

6.4% of all 3+ ratings were given to 'Our Polluted World'


### Are positive ratings more frequent before or after 1pm?
- _Assuming any ratings of 3 or above are considered positive_

In [7]:
with get_cursor(conn) as cursor:

    cursor.execute(
    f"""
    SELECT COUNT(r.numeric_value) as rate
    FROM exhibition AS e
    JOIN review AS rev
    ON e.exhibition_id = rev.exhibition_id
    JOIN rating AS r
    ON rev.rating_id = r.rating_id
    WHERE r.numeric_value in (3,4)
    AND EXTRACT(HOUR FROM rev.creation_date) < 13;
    """)

    before_one = ("before", cursor.fetchone()[0])

    cursor.execute(
    f"""
    SELECT COUNT(r.numeric_value)
    FROM exhibition AS e
    JOIN review AS rev
    ON e.exhibition_id = rev.exhibition_id
    JOIN rating AS r
    ON rev.rating_id = r.rating_id
    WHERE r.numeric_value in (3,4)
    AND EXTRACT(HOUR FROM rev.creation_date) > 13;
    """)

    after_one = ("after", cursor.fetchone()[0])
    answer = max(before_one, after_one, key=lambda x: x[1])[0]

print(f"There are {before_one[1]} positive ratings before 1pm, and {after_one[1]} after 1pm, so positive ratings are more frequent {answer} 1pm.")


There are 2835 positive ratings before 1pm, and 1029 after 1pm, so positive ratings are more frequent before 1pm.


### How many ratings each hour are above the average rating for exhibition 4?

In [8]:
with get_cursor(conn) as cursor:

    cursor.execute(
    f"""
    SELECT EXTRACT(HOUR FROM rev.creation_date) AS hour, COUNT(*) AS num_ratings
    FROM exhibition AS e
    JOIN review AS rev
    ON e.exhibition_id = rev.exhibition_id
    JOIN rating AS r
    ON rev.rating_id = r.rating_id
    JOIN(
        SELECT AVG(r.numeric_value) AS avg_rating_4
        FROM exhibition AS e
        JOIN review AS rev
        ON e.exhibition_id = rev.exhibition_id
        JOIN rating AS r
        ON rev.rating_id = r.rating_id
        WHERE e.exhibition_id = 4
        )
    AS avg_rating_4
    ON r.numeric_value > avg_rating_4.avg_rating_4
    GROUP BY hour
    ORDER BY hour;
    """)

    res = cursor.fetchall()

for row in res:
    print(f"There are {row[1]} ratings during the {row[0]}:00 hour that are above the average rating for exhibition 4.")

There are 1 ratings during the 7:00 hour that are above the average rating for exhibition 4.
There are 915 ratings during the 8:00 hour that are above the average rating for exhibition 4.
There are 998 ratings during the 9:00 hour that are above the average rating for exhibition 4.
There are 997 ratings during the 10:00 hour that are above the average rating for exhibition 4.
There are 961 ratings during the 11:00 hour that are above the average rating for exhibition 4.
There are 1039 ratings during the 12:00 hour that are above the average rating for exhibition 4.
There are 1034 ratings during the 13:00 hour that are above the average rating for exhibition 4.
There are 788 ratings during the 14:00 hour that are above the average rating for exhibition 4.
There are 510 ratings during the 15:00 hour that are above the average rating for exhibition 4.
There are 501 ratings during the 16:00 hour that are above the average rating for exhibition 4.
There are 2 ratings during the 17:00 hour t

### Do Zoology exhibitions get better ratings than other types?

In [9]:
#Find the average of all zoology exhibitions
# Compare to the average of all other exhibitions!

with get_cursor(conn) as cursor:

    cursor.execute(
        f"""
        SELECT d.department_name, AVG(r.numeric_value) AS average
        FROM exhibition AS e
        JOIN review AS rev
        ON e.exhibition_id = rev.exhibition_id
        JOIN rating AS r
        ON rev.rating_id = r.rating_id
        JOIN department AS d
        ON e.department_id = d.department_id
        GROUP BY d.department_id
        ORDER BY average DESC;
        """)
    
    result = cursor.fetchall()

other_exhibition_ratings = [row[1] for row in result if "Zoology" not in row[0]]
zoology_rating = [row[1] for row in result if row[0] == "Zoology"]
print(f"This is {zoology_rating > other_exhibition_ratings}")
for row in result:
    print(f"{row[0]} department exhibitions have an overall average rating of {round(row[1], 1)}")


This is False
Paleontology department exhibitions have an overall average rating of 2.9
Entomology department exhibitions have an overall average rating of 2.0
Geology department exhibitions have an overall average rating of 2.0
Zoology department exhibitions have an overall average rating of 1.9
Ecology department exhibitions have an overall average rating of 1.2


### What exhibition has the most emergencies?

In [10]:
with get_cursor(conn) as cursor:

    cursor.execute(
        f"""
        SELECT e.exhibition_name, count(ar.assistance_request_id)
        FROM exhibition AS e
        JOIN assistance_request AS ar
        ON e.exhibition_id = ar.exhibition_id
        JOIN assistance AS a
        ON ar.assistance_id = a.assistance_id
        WHERE a.numeric_value = 1
        GROUP BY e.exhibition_name
        ORDER BY count(ar.assistance_request_id) DESC
        LIMIT 1;
        """)

    result = cursor.fetchone()

print(f"The exhibition with the most emergency requests is {result[0]}, with {result[1]} requests")

The exhibition with the most emergency requests is The Crenshaw Collection, with 6 requests


### Which exhibitions receive fewer assistance request than the average?

In [11]:
with get_cursor(conn) as cursor:

    cursor.execute(
        f"""
        SELECT e.exhibition_name, COUNT(are.assistance_request_id)
        FROM exhibition AS e
        JOIN assistance_request AS are
        ON e.exhibition_id = are.exhibition_id
        GROUP BY e.exhibition_name
        HAVING COUNT(are.assistance_request_id) <
        (
        SELECT AVG(num_reqs)
        FROM (
            SELECT COUNT(ar.assistance_request_id) AS num_reqs
            FROM assistance_request AS ar
            JOIN exhibition AS e
            ON ar.exhibition_id = e.exhibition_id
            GROUP BY ar.exhibition_id
            )
        )
        """)
    
    result = cursor.fetchall()

for row in result:
    print(row[0])

Thunder Lizards
Measureless to Man
Adaptation


### Are there particular times when assistance requests/emergencies are more likely?

In [12]:
with get_cursor(conn) as cursor:

    cursor.execute(
        f"""
    SELECT EXTRACT(HOUR FROM ar.creation_date) AS hour, COUNT(*) AS num_ars
    FROM assistance_request AS ar
    GROUP BY hour
    ORDER BY num_ars DESC
    LIMIT 5;
    """)

    res = cursor.fetchall()


hours = [f"{int(row[0])}:00-{int(row[0])+1}:00" for row in res]

print(f"The five most common times for assistance requests to occur are:\n{
      "\n".join(hours)}")

The five most common times for assistance requests to occur are:
13:00-14:00
11:00-12:00
12:00-13:00
10:00-11:00
9:00-10:00
